# Анализ базы данных сервиса для чтения книг по подписке

Компания-стартапер купила крупный сервис для чтения книг по подписке. У нас в наличии база данных с информацией о книгах, издательствах, авторах, а также пользовательские обзоры книг.

**Цель исследования:** 
- Сформулировать ценностное предложение для нового продукта.

**Задача:** 
- Проанализировать базу данных

## Обзор данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**books** (данные о книгах)

In [2]:
query = '''

SELECT * 
FROM books 
LIMIT 5

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**authors** (данные об авторах)

In [3]:
query = '''

SELECT * 
FROM authors 
LIMIT 5

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**publishers** (данные об издательствах)

In [4]:
query = '''

SELECT * 
FROM publishers
LIMIT 5

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**ratings** (данные о пользовательских оценках книг)

In [5]:
query = '''

SELECT *
FROM ratings
LIMIT 5

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**reviews** (данные о пользовательских обзорах)

In [6]:
query = '''

SELECT * 
FROM reviews 
LIMIT 5

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Согласно документации к данным:

**books** (данные о книгах)
- `book_id` — идентификатор книги;         
- `author_id` — идентификатор автора;
- `title` — название книги;  
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги; 
- `publisher_id` — идентификатор издателя.

**authors** (данные об авторах)
- `author_id` — идентификатор автора;        
- `author` — имя автора.

**publishers** (данные об издательствах)
- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства.

**ratings** (данные о пользовательских оценках книг)
- `rating_id ` — идентификатор оценки;       
- `book_id ` —  идентификатор книги;
- `username ` — имя пользователя, оставившего оценку;
- `rating ` — оценка книги.

**reviews** (данные о пользовательских обзорах)
- `review_id  ` — идентификатор обзора;       
- `book_id  ` —  идентификатор книги;
- `username ` — имя автора обзора;
- `text  ` — текст обзора.

## Задания

### Посчитайте, сколько книг вышло после 1 января 2000 года.

In [7]:
query = '''

SELECT COUNT (book_id) AS books_cnt
FROM books
WHERE publication_date > '2000-01-01'

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,books_cnt
0,819


819 книг вышло после 1 января 2000 года.

### Для каждой книги посчитайте количество обзоров и среднюю оценку.

In [8]:
query = '''

SELECT b.book_id, 
       b.title, 
       COUNT(DISTINCT rev.review_id) AS reviews_cnt, 
       ROUND(AVG(rat.rating), 1) AS rating_avg
FROM books b

LEFT JOIN reviews rev
ON b.book_id = rev.book_id

LEFT JOIN  ratings rat
ON b.book_id = rat.book_id

GROUP BY b.book_id, b.title
ORDER BY reviews_cnt DESC, rating_avg DESC

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,book_id,title,reviews_cnt,rating_avg
0,948,Twilight (Twilight #1),7,3.7
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.4
2,656,The Book Thief,6,4.3
3,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.3
4,734,The Glass Castle,6,4.2
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.0
996,387,Leonardo's Notebooks,0,4.0
997,221,Essential Tales and Poems,0,4.0
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.7


В нашей базе данных 1000 книг. На каждую книгу приходится от 0 до 7 обзоров. Максимальное количество обзоров у книги Twilight (Twilight #1)	со средним рейтингом 3.7.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры.

In [9]:
query = '''

WITH tbl AS
(
SELECT publisher_id,
       book_id
FROM books
WHERE num_pages > 50
)

SELECT tbl.publisher_id,
       p.publisher,
       COUNT(tbl.book_id) OVER (PARTITION BY tbl.publisher_id) AS books_cnt
FROM tbl
LEFT JOIN publishers p ON p.publisher_id = tbl.publisher_id
ORDER BY books_cnt DESC
LIMIT 1


'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher,books_cnt
0,212,Penguin Books,42


Издательство, которое выпустило наибольшее количество книг (брошюры в расчет не берутся) - Penguin Books с 42 выпущенными книгами.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками.

In [10]:
query = '''

WITH tbl AS
(
SELECT DISTINCT book_id,
       COUNT(rating_id),
       AVG(rating) AS rating_avg
FROM ratings
GROUP BY book_id
HAVING COUNT(rating_id) >= 50
ORDER BY rating_avg DESC
LIMIT 1
)

SELECT a.author,
       tbl.rating_avg
FROM authors a
LEFT JOIN books b ON a.author_id = b.author_id
JOIN tbl ON tbl.book_id = b.book_id

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author,rating_avg
0,J.K. Rowling/Mary GrandPré,4.414634


Автор с самой высокой средней оценкой книг - J.K. Rowling/Mary GrandPré с рейтингом 4.414634. Здесь не учитывались книги, у которых меньше 50 оценок.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [11]:
query = '''
WITH tbl AS
(
SELECT DISTINCT username,
       COUNT(review_id) OVER (PARTITION BY username) AS reviews_count
FROM reviews
WHERE username IN (SELECT username
                   FROM ratings
                   GROUP BY username
                   HAVING COUNT(rating_id) > 48)
)

SELECT AVG (reviews_count) AS reviews_count_avg
FROM tbl

'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,reviews_count_avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, составляет 24 обзора.